# DataFrame Library Comparison: Polars vs Pandas vs Dask

**Real-world performance test on event extraction from market data**

This notebook compares three popular DataFrame libraries on a real data engineering task:
extracting trading events from market order snapshots.

## The Task

Event extraction requires:
1. **Reading Parquet files** (I/O performance)
2. **Filtering data** by conditions
3. **Joining snapshots** on order_id
4. **Detecting changes** (disappeared orders, volume changes, new orders)

## The Libraries

- **Polars**: Modern, multi-threaded, built on Apache Arrow
- **Pandas**: Classic, single-threaded, most widely used
- **Dask**: Parallel/distributed computing, pandas-compatible API

Let's see which one performs best!

---

## Configuration

In [1]:
# ============================================================
# CONFIGURATION
# ============================================================

from pathlib import Path

# Data location
EVE_DATA_PATH = Path("/Users/fred/Projects/Gaming/Eve Online/eve_market_data/data/orders")

# How many snapshot pairs to test (more = longer test, more accurate)
NUM_SNAPSHOT_PAIRS = 3  # Use 3 for quick test, 11 for full hour

# Backends to test
BACKENDS = ['polars', 'pandas', 'dask']

print("📋 Configuration:")
print(f"   Data path: {EVE_DATA_PATH}")
print(f"   Testing {NUM_SNAPSHOT_PAIRS} snapshot pairs")
print(f"   Backends: {', '.join(BACKENDS)}")

📋 Configuration:
   Data path: /Users/fred/Projects/Gaming/Eve Online/eve_market_data/data/orders
   Testing 3 snapshot pairs
   Backends: polars, pandas, dask


## Setup

Import libraries and load snapshot files.

In [ ]:
import sys
import time
from datetime import datetime
import polars as pl
import pandas as pd
import dask.dataframe as dd

print("✅ Imports successful")

✅ Imports successful


In [3]:
# Find snapshot files
snapshot_files = sorted(EVE_DATA_PATH.glob("region_10000002_2025-10-22T07-*.parquet"))
snapshot_files += sorted(EVE_DATA_PATH.glob("region_10000002_2025-10-22T08-*.parquet"))[:3]
snapshot_files = snapshot_files[:12]

print(f"📂 Found {len(snapshot_files)} snapshot files")
print(f"\n⏱️  Time range:")
print(f"   Start: {snapshot_files[0].name}")
print(f"   End:   {snapshot_files[-1].name}")

# Create snapshot pairs for testing
snapshot_pairs = [
    (snapshot_files[i], snapshot_files[i+1]) 
    for i in range(min(NUM_SNAPSHOT_PAIRS, len(snapshot_files)-1))
]

print(f"\n🔍 Will test with {len(snapshot_pairs)} snapshot pairs")

📂 Found 12 snapshot files

⏱️  Time range:
   Start: region_10000002_2025-10-22T07-00-00+00-00.parquet
   End:   region_10000002_2025-10-22T07-55-00+00-00.parquet

🔍 Will test with 3 snapshot pairs


---

## Performance Test: Real Event Extraction with Initialization

We'll run the exact same event detection logic with three different DataFrame backends.

### Complete Event Extraction Process:

**Step 1: Initialization** (create baseline state)
- Read first snapshot (snapshot "0")
- Generate ORDER_OPENED events for all existing orders
- This establishes the baseline state, making the event log self-contained

**Step 2: Delta Event Detection** (process subsequent snapshots)
1. Read two consecutive snapshots (Parquet files)
2. Identify disappeared orders (→ TRADE or CANCELLED events)
3. Detect volume reductions (→ TRADE events)
4. Find new orders (→ ORDER_OPENED events)
5. Spot price changes (→ PRICE_CHANGED events)

This is a real-world workload combining I/O, joins, filters, and transformations.

**Why initialization matters**: With the baseline events from snapshot "0", we can reconstruct the complete order book state at any timestamp using only the event log - no anchor snapshots needed!

In [4]:
print(f"🔍 Comparing OPTIMIZED event extraction performance")
print(f"   Testing with {len(snapshot_pairs)} snapshot pairs\n")
print("="*70)

# Import optimized detectors
from src.event_extractor.event_detector_polars import PolarsEventDetector
from src.event_extractor.event_detector_pandas import PandasEventDetector
from src.event_extractor.event_detector_dask import DaskEventDetector

results = {}

# Test 1: POLARS (optimized - vectorized operations)
print("\n📊 POLARS (vectorized, multi-threaded)")
detector_polars = PolarsEventDetector()

start = time.time()

# Step 1: Initialize from first snapshot (baseline state)
timestamp_0 = datetime.strptime(snapshot_files[0].stem.split('_')[-1].replace('+00-00', ''), "%Y-%m-%dT%H-%M-%S")
init_events = detector_polars.initialize_from_snapshot(snapshot_files[0], timestamp_0)
total_events = len(init_events)

# Step 2: Process delta events from snapshot pairs
for prev_file, curr_file in snapshot_pairs:
    timestamp_str = curr_file.stem.split('_')[-1].replace('+00-00', '')
    timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H-%M-%S")
    events = detector_polars.detect_events(prev_file, curr_file, timestamp)
    total_events += len(events)

elapsed = time.time() - start
results['polars'] = {
    'time': elapsed,
    'events': total_events,
    'init_events': len(init_events),
    'delta_events': total_events - len(init_events),
    'avg_per_interval': elapsed / len(snapshot_pairs)
}

print(f"   Time: {elapsed:.3f} seconds")
print(f"   Events: {total_events:,} ({len(init_events):,} init + {total_events - len(init_events):,} delta)")
print(f"   Avg per interval: {elapsed/len(snapshot_pairs):.3f}s")

# Test 2: PANDAS (optimized - vectorized groupby, itertuples)
print("\n📊 PANDAS (vectorized where possible, single-threaded)")
detector_pandas = PandasEventDetector()

start = time.time()

# Step 1: Initialize from first snapshot
init_events = detector_pandas.initialize_from_snapshot(snapshot_files[0], timestamp_0)
total_events = len(init_events)

# Step 2: Process delta events
for prev_file, curr_file in snapshot_pairs:
    timestamp_str = curr_file.stem.split('_')[-1].replace('+00-00', '')
    timestamp = datetime.strptime(timestamp_str, "%Y-%m-%dT%H-%M-%S")
    events = detector_pandas.detect_events(prev_file, curr_file, timestamp)
    total_events += len(events)

elapsed = time.time() - start
results['pandas'] = {
    'time': elapsed,
    'events': total_events,
    'init_events': len(init_events),
    'delta_events': total_events - len(init_events),
    'avg_per_interval': elapsed / len(snapshot_pairs)
}

print(f"   Time: {elapsed:.3f} seconds")
print(f"   Events: {total_events:,} ({len(init_events):,} init + {total_events - len(init_events):,} delta)")
print(f"   Avg per interval: {elapsed/len(snapshot_pairs):.3f}s")

# Test 3: DASK (batch processing - TRUE parallelism)
print("\n📊 DASK (parallel batch processing across CPU cores)")
detector_dask = DaskEventDetector(n_workers=4)

start = time.time()

# Step 1: Initialize from first snapshot
init_events = detector_dask.initialize_from_snapshot(snapshot_files[0], timestamp_0)

# Step 2: Prepare batch data for parallel processing
batch_pairs = [
    (prev, curr, datetime.strptime(curr.stem.split('_')[-1].replace('+00-00', ''), "%Y-%m-%dT%H-%M-%S"))
    for prev, curr in snapshot_pairs
]

# Process all pairs in parallel
delta_events = detector_dask.detect_events_batch(batch_pairs)

elapsed = time.time() - start
total_events = len(init_events) + len(delta_events)

results['dask'] = {
    'time': elapsed,
    'events': total_events,
    'init_events': len(init_events),
    'delta_events': len(delta_events),
    'avg_per_interval': elapsed / len(snapshot_pairs)
}

print(f"   Time: {elapsed:.3f} seconds")
print(f"   Events: {total_events:,} ({len(init_events):,} init + {len(delta_events):,} delta)")
print(f"   Avg per interval: {elapsed/len(snapshot_pairs):.3f}s")
print(f"   (Processing {len(snapshot_pairs)} pairs in parallel!)")

print("\n" + "="*70)

🔍 Comparing OPTIMIZED event extraction performance
   Testing with 3 snapshot pairs


📊 POLARS (vectorized, multi-threaded)
🌱 Initializing event log from snapshot: 375,338 orders
   Time: 1.142 seconds
   Events: 378,250 (375,338 init + 2,912 delta)
   Avg per interval: 0.381s

📊 PANDAS (vectorized where possible, single-threaded)
🌱 Initializing event log from snapshot: 375,338 orders
   Time: 2.257 seconds
   Events: 378,250 (375,338 init + 2,912 delta)
   Avg per interval: 0.752s

📊 DASK (parallel batch processing across CPU cores)
🌱 Initializing event log from snapshot: 375,338 orders
   Time: 2.112 seconds
   Events: 378,250 (375,338 init + 2,912 delta)
   Avg per interval: 0.704s
   (Processing 3 pairs in parallel!)



---

## Results Analysis

In [5]:
# Sort backends by performance (fastest to slowest)
sorted_backends = sorted(BACKENDS, key=lambda b: results[b]['time'])
winner = sorted_backends[0]
pandas_time = results['pandas']['time']
winner_time = results[winner]['time']

print("\n⚡ SPEEDUP COMPARISON (vs Pandas):")
print("="*70)

for backend in sorted_backends:
    if backend != 'pandas':
        speedup = pandas_time / results[backend]['time']
        print(f"   {backend.capitalize():8} {speedup:5.2f}× faster than Pandas")
    else:
        print(f"   {backend.capitalize():8} baseline")

print("\n" + "="*70)

# Show detailed breakdown - ordered by performance
print("\n📊 DETAILED BREAKDOWN (fastest → slowest):")
print("="*70)
print(f"{'Rank':<6} {'Backend':<10} {'Total Time':>12} {'Avg/Interval':>15} {'Events':>10}")
print("-"*70)

for rank, backend in enumerate(sorted_backends, 1):
    r = results[backend]
    medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉"
    print(f"{medal} {rank}. {backend.capitalize():<10} {r['time']:>10.3f}s {r['avg_per_interval']:>13.3f}s {r['events']:>10,}")

print("="*70)


⚡ SPEEDUP COMPARISON (vs Pandas):
   Polars    1.98× faster than Pandas
   Dask      1.07× faster than Pandas
   Pandas   baseline


📊 DETAILED BREAKDOWN (fastest → slowest):
Rank   Backend      Total Time    Avg/Interval     Events
----------------------------------------------------------------------
🥇 1. Polars          1.142s         0.381s    378,250
🥈 2. Dask            2.112s         0.704s    378,250
🥉 3. Pandas          2.257s         0.752s    378,250
